# Comparison
    
On the internet there are plenty of articles on how to predict a stocks price using LSTMs. I'll perform an example of that here to compare against later

In [2]:
import pandas as pd
import numpy as np
import math
import random
from datetime import date, timedelta

from sklearn.preprocessing import StandardScaler as SS
from sklearn.metrics import mean_squared_error as MSE


from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Nadam

import talos

np.seed = 42

In [41]:
df = pd.read_csv('change.csv', index_col=0)
df.head()
ss = SS()
s_df = pd.DataFrame(ss.fit_transform(df), columns=df.columns, index=df.index)

In [42]:
s_df.head()

,A,AA,AAIC,AAN,AAP,AAT,AB,ABB,ABBV,ABC,...,ZBH,ZEN,ZEPP,ZIM,ZNH,ZTO,ZTR,ZTS,ZUO,ZYME
1999-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-02,NaN,NaN,1.295591,NaN,NaN,NaN,1.844412,NaN,NaN,-0.126726,...,NaN,NaN,NaN,NaN,-0.198055,NaN,0.235925,NaN,NaN,NaN
1999-11-03,NaN,NaN,0.441830,NaN,NaN,NaN,0.388344,NaN,NaN,-0.295364,...,NaN,NaN,NaN,NaN,0.111521,NaN,-0.018540,NaN,NaN,NaN
1999-11-04,NaN,NaN,0.003413,NaN,NaN,NaN,0.093490,NaN,NaN,0.012077,...,NaN,NaN,NaN,NaN,0.422628,NaN,-0.018540,NaN,NaN,NaN
1999-11-05,NaN,NaN,1.018696,NaN,NaN,NaN,0.482988,NaN,NaN,-0.076134,...,NaN,NaN,NaN,NaN,0.279128,NaN,-0.018540,NaN,NaN,NaN


In [53]:
s_df = s_df.loc['2004-11-01':'2018-11-01']
syms = s_df.columns[df.notna().sum() > 1200]


In [71]:
lstm_data = {}
syms = syms[:40]
for sym in syms:
    
    sym_df = s_df[sym].dropna()
    
    X = []
    y = []

    for j in range(90, len(sym_df)-1):
        X.append(sym_df.iloc[j-90:j])
        y.append(sym_df.iloc[j])
    lstm_data[sym] = (X, y)

In [60]:
model2 = Sequential()

model2.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model2.add(Dropout(0.2))

model2.add(LSTM(units = 50, return_sequences = True))
model2.add(Dropout(0.2))

model2.add(LSTM(units = 50, return_sequences = True))
model2.add(Dropout(0.2))

model2.add(LSTM(units = 50))
model2.add(Dropout(0.2))

model2.add(Dense(units = 1))

model2.compile(optimizer = 'adam', loss = 'mean_squared_error')

mc2 = ModelCheckpoint(f'models/comp_b{i}.h5', save_best_only=True)
cb2 = cb = [mc2, es]

In [ ]:
results = {}
for sym in syms:
    
    X = np.array(lstm_data[sym][0])
    y = np.array(lstm_data[sym][1])
    test_len = 120
    ind_val = np.random.choice(len(X)-test_len, int(len(X)/10), replace=False)
    ind_train = [x for x in range(len(X)-test_len) if x not in ind_val]
    X_train = X[ind_train]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
    y_train = y[ind_train]
    X_val = X[ind_val]
    X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1],1))
    y_val = y[ind_val]
    X_test = X[-test_len:]
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
    y_test = y[-test_len:]
    
    history2 = model2.fit(X_train, y_train, epochs = 30, batch_size = 160, validation_data=(X_val, y_val), callbacks=cb2)
    min_lstm = min(history2.history['val_loss'])
    print(min_lstm)

    
    y_pred = [x[-1] for x in X]
    base = MSE(y, y_pred)
    print('Baseline:', base)
    
    results[sym] = {'Baseline': base,
                  'LSTM': min_lstm
                 }
    
    

Epoch 1/30
19/19 [==============================] - 34s 1s/step - loss: 0.2535 - val_loss: 0.1704
Epoch 2/30
19/19 [==============================] - 23s 1s/step - loss: 0.2516 - val_loss: 0.1723
Epoch 3/30
19/19 [==============================] - 25s 1s/step - loss: 0.2513 - val_loss: 0.1734
Epoch 4/30
19/19 [==============================] - 25s 1s/step - loss: 0.2485 - val_loss: 0.1744
Epoch 5/30
19/19 [==============================] - 41s 2s/step - loss: 0.2489 - val_loss: 0.1731
Epoch 6/30
19/19 [==============================] - 26s 1s/step - loss: 0.2489 - val_loss: 0.1724
Epoch 7/30
19/19 [==============================] - 20s 1s/step - loss: 0.2495 - val_loss: 0.1757
Epoch 8/30
19/19 [==============================] - 21s 1s/step - loss: 0.2483 - val_loss: 0.1726
Epoch 9/30
19/19 [==============================] - 21s 1s/step - loss: 0.2492 - val_loss: 0.1735
Epoch 10/30
19/19 [==============================] - 20s 1s/step - loss: 0.2485 - val_loss: 0.1742
Epoch 11/30
19/19 [

19/19 [==============================] - 21s 1s/step - loss: 0.8184 - val_loss: 0.9183
Epoch 9/30
19/19 [==============================] - 20s 1s/step - loss: 0.8037 - val_loss: 0.9123
Epoch 10/30
19/19 [==============================] - 21s 1s/step - loss: 0.8007 - val_loss: 0.9394
Epoch 11/30
19/19 [==============================] - 23s 1s/step - loss: 0.8144 - val_loss: 0.9202
Epoch 12/30
19/19 [==============================] - 21s 1s/step - loss: 0.7892 - val_loss: 0.8922
Epoch 13/30
19/19 [==============================] - 20s 1s/step - loss: 0.7829 - val_loss: 0.8999
Epoch 14/30
19/19 [==============================] - 21s 1s/step - loss: 0.7629 - val_loss: 0.8699
Epoch 15/30
19/19 [==============================] - 22s 1s/step - loss: 0.7532 - val_loss: 0.9439
Epoch 16/30
19/19 [==============================] - 21s 1s/step - loss: 0.7604 - val_loss: 0.9658
Epoch 17/30
19/19 [==============================] - 21s 1s/step - loss: 0.7727 - val_loss: 0.9470
Epoch 18/30
19/19 [====

In [66]:
#Baseline: use the previous day's value as the prediction
for sym in syms:
    
    sym_df = s_df[sym].dropna()
    
    y_pred = []
    y = []

    for j in range(90, len(sym_df)-1):
        y_pred.append(sym_df.iloc[j-1])
        y.append(sym_df.iloc[j])
    lstm_data[sym] = (X, y)
    
    print(MSE(y, y_pred))

0.477777903282233
0.6263179438281827
1.6479413443231126
0.8463135238155464
1.4333049467971295
1.8027309351257212
1.2770253322177527
1.4025808121079044
2.196524227399131
0.7633858303402123
1.2534029045367507
0.8508085491230765
0.6533323277213866
0.5620349872405065
2.093357467546747


In [73]:
results

{0: {'Baseline': 2.093357467546747, 'LSTM': 0.44568097591400146}}